# Evaluation of accuracy of the transcripts

This notebook is designed to assess the performance of predictions after manual verification. *(Also providing enhanced visual HTML files to help interpret metrics clearly.)*

- After manually verifying transcripts, you can use the verified transcripts as reference (ground truth) data to test the accuracy of various models.
    
- **However**, it's essential to note that if the reference file (ground truth) was created by using a model’s predictions as a template the ground truth may carry a bias towards that initial model. *For instance, if a model omitted interruptions or adjusted sentence structure and the verifier found these changes accurate in context, these modifications might remain, unintentionally favoring that model.*

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import shutil
import os

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from utils.evaluation_helpers import process_folder_csv, load_data_time, compute_der, process_folder_text
from utils.format_helpers import get_files

# Predictions vs References: Performance evaluation (ASR & Diarization)

### Preparing evaluation folder (Predictions vs References)

Useful if want to test/compare differents models.

In [3]:
directories = ['../results/Compassion', '../results/OBE1', '../results/OBE2'] # The csv file that you want to compare
pred_folder = '../evaluation/predictions' # Specify the name if trying to compare different ASR/Diarization models.

def copy_csv_files(directories, pred_folder):
    pred_files = []
    for directory in directories:
        pred_files.append(get_files(directory, 'csv'))
    pred_files = [item for sublist in pred_files for item in sublist]

    os.makedirs(pred_folder, exist_ok=True)
    for file in pred_files:
        shutil.copy(file, pred_folder)

copy_csv_files(directories, pred_folder)

## CSV Files

When corrections are made directly in the CSV file and timestamps are adjusted, it ensures a more robust computation of the DER (Diarization Error Rate) metric. However, for simplicity and efficiency during the verification process, the **Text File** method described below is recommended.

### ASR and Diarization metrics + Visual Tool to easily understand missmatch

In [ ]:
prediction_folder = "../evaluation/predictions"
reference_folder = "../evaluation/references"

# Provide the original duration of the audio files.
# This ensures the diarization error rate (DER) metric accounts for the total audio duration, not just the speech segments detected by the model.
# If this information is missing, DER calculations may be inaccurate as they consider only the detected speech intervals, worsening the DER metric.
orignal_info = pd.read_csv("./outputs/audio_data.csv")

metric = process_folder_csv(prediction_folder, reference_folder, info=orignal_info)
metric.to_csv('../evaluation/metrics.csv', index=False)

### (Optional) Dialogue DER Analysis in CSV instead than in HTML

In [6]:
def dialogue_DER(reference_file, prediction_file, output_file=None):
    df_ref, df_pred = load_data_time(reference_file, prediction_file)
    dialogue_df, error_durations = compute_der(df_ref, df_pred)

    # Display error durations and DER
    print("\nError Durations and DER:")
    for key, value in error_durations.items():
        if key == 'DER':
            print(f"{key}: {value:.2%}")
        else:
            print(f"{key}: {value:.3f} seconds")

    if output_file:
        dialogue_df.to_csv(output_file, index=False)

In [ ]:
reference_file = "../evaluation/references/S301final.csv"
prediction_file = "../evaluation/predictions/S301final.csv"
output_file = '../outputs/dia_S301final.csv'

dialogue_DER(reference_file, prediction_file)#, output_file)

# Text Files 
When corrections are made directly in a text file rather than adjusting timestamps in a CSV (as was done for the Grief dataset interviews), a simplified approach to DER is required:

Since time segments are not adjusted, we ignore missed speech and false alarm errors. Instead, we focus solely on confusion errors, defined as the number of words reassigned from one speaker to another. *This replaces the traditional time-based DER calculations with a straightforward, text-based method.*

**Note:** This approach can be preferred as it accelerates the manual correction process by providing a better visual representation, with segments concatenated by turn in the text file and by neglecting the need to correct timestamps, it offers significant time savings without compromising the core analysis.

**Assumptions:**

- During manual verification, brackets (``[]``) are exclusively used to denote speaker labels and are not added for any other purpose.
- Speaker names/labels within the brackets remain consistent throughout the corrections.

These assumptions ensure the algorithm can process the text reliably while maintaining the integrity of speaker assignments.

In [ ]:
prediction_folder = "../evaluation/predictions_text"
reference_folder = "../evaluation/references_text"

metric = process_folder_text(prediction_folder, reference_folder,dir_visual="meditation_visual_text")
metric.to_csv('../evaluation/meditation_metrics_text.csv', index=False)

# Visualize performance metrics

## Meditations Visualisation

In [ ]:
# Create a mapping of Filename to Experiment
df_audio = pd.read_csv('./outputs/audio_data.csv')
filename_to_experiment = dict(zip(df_audio['File Name'],df_audio['Experiment']))

df = pd.read_csv('../evaluation/metrics.csv')
df['Experiment'] = df['Filename'].map(filename_to_experiment)
df

In [10]:
def plot_boxplots(df, title=None, file_name=None):
    """
    Plots box plots for WER and DER.
    
    Args:
        df (pd.DataFrame): DataFrame containing the metrics.
    """
    plt.figure(figsize=(8, 4))
    
    # Melt the DataFrame for easier plotting with Seaborn
    melted_df = df.melt(id_vars='Filename', value_vars=['WER', 'DER'], var_name='Metric', value_name='Value')
    ax = sns.boxplot(x='Metric', y='Value',hue="Metric" ,data=melted_df,)# palette=['skyblue', 'salmon'])
    plt.ylabel('Error Rate')
    plt.xlabel('')
    if title:
        plt.title(title)
    
    # Calculate and print medians
    medians = melted_df.groupby('Metric')['Value'].median()
    print("Median Values:")
    for metric, median_value in medians.items():
        print(f"  {metric}: {median_value:.3f}")
    
    plt.tight_layout()
    if file_name:
        plt.savefig(file_name, dpi=600)
    plt.show()

In [ ]:
plot_boxplots(df, file_name='images/meditation_metrics_time.png',title='Meditation with time-stamps DER')

In [12]:
def plot_boxplot_experiment(df):
    """
    Plots box plots for WER and DER, grouped by Experiment.
    
    Args:
        df (pd.DataFrame): DataFrame containing the metrics and Experiment column.
    """
    plt.figure(figsize=(10, 6))
    
    # Melt the DataFrame for easier plotting with Seaborn
    melted_df = df.melt(id_vars=['Filename', 'Experiment'], 
                        value_vars=['WER', 'DER'], 
                        var_name='Metric', 
                        value_name='Value')
    
    # Create the boxplot with Experiment as hue
    sns.boxplot(x='Metric', y='Value', hue='Experiment', data=melted_df, palette='pastel')
    sns.stripplot(x='Metric', y='Value', hue='Experiment', data=melted_df, size=4, linewidth=1,dodge=True, jitter=True, legend=False, edgecolor="k")
    
    plt.ylabel('Error Rate')
    plt.xlabel('')
    #plt.title('Boxplots of WER and DER by Experiment')
    plt.legend(title='Experiment')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_boxplot_experiment(df)

## Grief & Meditation Comparaison

In [14]:
df_g = pd.read_csv('../evaluation/grief_metrics_text.csv')
df_m = pd.read_csv('../evaluation/med_metrics_text.csv')

In [ ]:
df_g["Group"] = "Grief"
df_m["Group"] = "Meditation"

# Concatenate the two DataFrames
df_text = pd.concat([df_m, df_g], ignore_index=True)
df_text

In [ ]:
plot_boxplots(df_g, title="Grief Text", file_name="images/grief_text_metrics.png")

In [ ]:
plot_boxplots(df_m,title="Meditation Text", file_name="images/meditation_text_metrics.png")

In [18]:
def plot_boxplots_group(df, tittle=None, file_name=None):
    """
    Plots two boxplots in subplots for WER and DER, grouped by the 'Group' column.
    
    Args:
        df (pd.DataFrame): DataFrame containing the metrics.
    """
    # Melt the DataFrame for easier plotting with Seaborn
    melted_df = df.melt(
        id_vars=['Filename', 'Group'], 
        value_vars=['WER', 'DER'], 
        var_name='Metric', 
        value_name='Value',
    )
    
    # Calculate medians grouped by Metric and Group
    medians = melted_df.groupby(['Metric', 'Group'])['Value'].median()
    print("Median Values:")
    print("-" * 50)
    print(medians.to_string())

    
    # Create the subplots
    fig, axes = plt.subplots(2, 1, figsize=(8, 10), sharey=False)
    
    # Plot WER boxplot
    sns.boxplot(
        ax=axes[0],
        x='Group', y='Value', 
        data=melted_df[melted_df['Metric'] == 'WER'], 
        hue='Group'
    )
    axes[0].set_title('WER by Dataset')
    axes[0].set_ylabel('WER')
    axes[0].set_xlabel('')
    
    # Plot DER boxplot
    sns.boxplot(
        ax=axes[1],
        x='Group', y='Value', 
        data=melted_df[melted_df['Metric'] == 'DER'], 
        hue='Group'
    )
    axes[1].set_title('DER by Dataset')
    axes[1].set_ylabel('DER')
    axes[1].set_xlabel('')

    if tittle:
        fig.suptitle(tittle, fontsize=16)
    
    # Adjust layout
    plt.tight_layout()
    if file_name:
        plt.savefig(file_name, dpi=600)
    plt.show()

In [ ]:
plot_boxplots_group(df_text, file_name="images/comparaison_Grief_Meditation.png")